In [3]:
import matplotlib.pyplot as plt
import time
import serial
import csv
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D
from datetime import datetime

file_count = input("測定回数を入力してください(例:1): ")
measurement_kind = input("測定するマッキベンの種類を入力してください(4s or 4l or 6 or s or u): ")
pressure = input("圧力を入力してください(例:4): ")

pressure = f"0.{pressure}"  

start_time = 10
end_time = 14.4

測定回数を入力してください(例:1):  tes
測定するマッキベンの種類を入力してください(4s or 4l or 6 or s or u):  tes
圧力を入力してください(例:4):  trs


In [6]:
# com8 からの圧力データを取得する設定
com8 = serial.Serial("COM16", 115200)
# com15 からのセンサーデータを取得する設定
com15 = serial.Serial('COM13', 115200, timeout=1)

path = 'C:\\Users\\Mizuki\\University\\Reserch\\length_sensor\\JupyterLab\\data\\pressure_control\\input\\'
path2 = 'C:\\Users\\Mizuki\\University\\Reserch\\length_sensor\\JupyterLab\\data\\pressure_control\\output\\'

file_name_all_com8 = f"{datetime.now().strftime('%Y%m%d')}_{file_count}_{measurement_kind}_{pressure}_all_p_input"
file_name_partial_com8 = f"{datetime.now().strftime('%Y%m%d')}_{file_count}_{measurement_kind}_{pressure}_partial_p_input"
date_com8 = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

file_name_all_com15 = f"{datetime.now().strftime('%Y%m%d')}_{file_count}_{measurement_kind}_{pressure}_all_p_output"
file_name_partial_com15 = f"{datetime.now().strftime('%Y%m%d')}_{file_count}_{measurement_kind}_{pressure}_partial_p_output"
date_com15 = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

print('start!')

t0_com8 = time.time()
t0_com15 = time.time()

# データを保存するリスト
all_time_vals_com8 = []
all_p_vals_com8 = []
partial_time_vals_com8 = []
partial_p_vals_com8 = []
partial_start_time_com8 = None

all_time_vals_com15 = []
all_length_vals_com15 = []  
all_p_vals_com15 = []
all_force_vals_com15 = []
partial_time_vals_com15 = []
partial_length_vals_com15 = []  
partial_p_vals_com15 = []
partial_force_vals_com15 = []
partial_start_time_com15 = None

with open(path + file_name_all_com8 + ".csv",mode="w", newline="") as all_f_com8, \
        open(path + file_name_partial_com8 + ".csv",mode="w", newline="") as partial_f_com8, \
        open(path2 + file_name_all_com15 + ".csv",mode="w", newline="") as all_f_com15, \
        open(path2 + file_name_partial_com15 + ".csv",mode="w", newline="") as partial_f_com15:
    
    # print("time,p,voltage", file=all_f_com8)
    # print("time,p,voltage", file=partial_f_com8)
    # print("time,length,pressure,force", file=all_f_com15)
    # print("time,length,pressure,force", file=partial_f_com15)

    try:
        while True:
            # com8 からのデータ処理
            writer = csv.writer(all_f_com8)
            writer.writerow(["Time(s)","Pressure(MPa)", "voltage(V)"])
            
            t_com8 = time.time()
            t_inner_com8 = t_com8 - t0_com8
            if t_inner_com8 > 15:
                print("saved as " + file_name_all_com8 + ".csv" + "!")
                break
                
            
            data_com8 = com8.readline().decode().strip()
            current_time_com8 = time.time()
            elapsed_time_com8 = round((current_time_com8 - t0_com8), 3)  # 四捨五入して小数点以下第三位まで

            print(f"{elapsed_time_com8} - {data_com8}")

            if data_com8 and not data_com8.startswith("Sensor reading stopped"):
                parts_com8 = data_com8.split(',')
                if len(parts_com8) == 3:
                    current_time, p_val_com8, voltage = parts_com8
                    p_val_com8 = float(p_val_com8) / 10.0  # bar から MPa への変換
                    writer.writerow([current_time, p_val_com8, voltage])

                    if t_inner_com8 >= start_time and t_inner_com8 <= end_time:
                        if partial_start_time_com8 is None:
                            partial_start_time_com8 = elapsed_time_com8
                        writer.writerow([elapsed_time_com8- partial_start_time_com8,p_val_com8, voltage])
                        
                else:
                    print("Invalid data format:", data_com8)
            elif data_com8.startswith("Sensor reading stopped"):
                print("Sensor reading stopped. Exiting loop.")
                break

            # com15 からのデータ処理
            t_com15 = time.time()
            t_inner_com15 = t_com15 - t0_com15
            if t_inner_com15 > 15:
                print("saved as " + file_name_all_com15 + ".csv" + "!")
                break

            data_com15 = com15.readline().decode().strip()
            current_time_com15 = time.time()
            elapsed_time_com15 = round((current_time_com15 - t0_com15), 3)  # 四捨五入して小数点以下第三位まで

            print(f"{elapsed_time_com15} - {data_com15}")

            if data_com15 and not data_com15.startswith("Sensor reading stopped"):
                parts_com15 = data_com15.split(',')
                if len(parts_com15) == 3:
                    wire_length, p_val_com15, force = parts_com15
                    p_val_com15 = float(p_val_com15) / 1000.0  

                    all_time_vals_com15.append(elapsed_time_com15)
                    if wire_length:
                        all_length_vals_com15.append(float(wire_length))  # 追加した行
                    all_p_vals_com15.append(p_val_com15)
                    if force:
                        all_force_vals_com15.append(float(force))  # 追加した行

                    print("{},{},{},{}".format(elapsed_time_com15, wire_length, p_val_com15, force), file=all_f_com15)

                    if t_inner_com15 >= start_time and t_inner_com15 <= end_time:
                        if partial_start_time_com15 is None:
                            partial_start_time_com15 = elapsed_time_com15
                        partial_time_vals_com15.append(elapsed_time_com15 - partial_start_time_com15)
                        partial_length_vals_com15.append(float(wire_length))  # 追加した行
                        partial_p_vals_com15.append(p_val_com15)
                        partial_force_vals_com15.append(float(force))

                        print("{},{},{},{}".format(elapsed_time_com15, wire_length, p_val_com15, force), file=partial_f_com15)
                else:
                    print("Invalid data format:", data_com15)
            elif data_com15.startswith("Sensor reading stopped"):
                print("Sensor reading stopped. Exiting loop.")
                break


    except KeyboardInterrupt:
        print("Interrupt received. Saving CSV files and exiting.")

    finally:
        com8.close()
        com15.close()


start!
3.909 - 3.00,-0.03,10.00
3.916 - 1,0.00
Invalid data format: 1,0.00
3.918 - 3.01,-0.03,0.00
3.918 - 60.20,0,0.00
3.936 - 3.02,-0.03,0.00
3.936 - 60.20,0,0.00
3.951 - 3.04,-0.02,0.00
3.951 - 60.20,0,0.00
3.965 - 3.05,-0.03,0.00
3.966 - 60.20,-1,0.00
3.966 - 3.06,-0.03,0.00
3.966 - 60.20,0,0.00
3.984 - 3.07,-0.03,0.00
3.984 - 60.20,0,0.00
4.0 - 3.08,-0.03,0.00
4.0 - 60.20,0,0.00
4.0 - 3.10,-0.02,0.00
4.0 - 60.20,0,0.00
4.018 - 3.11,-0.03,0.00
4.027 - 60.20,-1,0.00
4.038 - 3.12,-0.03,0.00
4.039 - 60.20,0,0.00
4.047 - 3.13,-0.03,0.00
4.048 - 60.20,0,0.00
4.048 - 3.14,-0.03,0.00
4.048 - 60.20,0,0.00
4.072 - 3.16,-0.02,0.00
4.073 - 60.20,-1,0.00
4.085 - 3.17,-0.03,0.00
4.086 - 60.20,-1,0.00
4.097 - 3.18,-0.03,0.00
4.098 - 60.20,-1,0.00
4.098 - 3.19,-0.02,0.00
4.098 - 60.20,-1,0.00
4.115 - 3.20,-0.03,0.00
4.115 - 60.20,-1,0.00
4.133 - 3.22,-0.02,0.00
4.135 - 60.20,-1,0.00
4.144 - 3.23,-0.03,0.00
4.146 - 60.20,-1,0.00
4.151 - 3.24,-0.03,0.00
4.151 - 60.20,-1,0.00
4.169 - 3.25,-0.03,0.00